# Notebook to generate track from waypoints
#### How to get Waypoints ?
* Print the waypoints array in the reward function and train the model for 5 min or less.
* Download the logs and define the waypoints as an array with the name `waypoints` and save the file to `TRACK_NAME.py` in the folder `Tracks`.
* Refer `stratus_waypoints.py`.

#### How to get Track Width ?
* When you evaluate a model in the console, we get list of all tracks with length and width for each track. Get width from there in meters.

In [ ]:
import glob
import numpy as np
from shapely.geometry import Point, Polygon
from shapely.geometry.polygon import LinearRing, LineString
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sys,os

In [ ]:
# List available waypoints files to analyze
available_track_files = glob.glob("../Tracks/**.py")
available_track_names = list(map(lambda x: os.path.basename(x).split('.npy')[0], available_track_files))
available_track_names

## Configure the Waypoints File and Width of the Track

In [ ]:
#Configuration
TRACK_NAME="stratus"
WAYPOINT_FILE_NAME = TRACK_NAME + '_waypoints'
TRACK_WIDTH = 1.07

sys.path.append(os.path.abspath('../Tracks'))
wp = importlib.import_module( 'stratus_waypoints', package=None)
waypoints = wp.waypoints
print("No. of waypoints loaded : ",len(waypoints))

In [ ]:
def plot_coords(ax, ob):                                                        
    x, y = ob.xy                                                                
    ax.plot(x, y, '.', color='#999999', zorder=1)                               
                                                                                
def plot_bounds(ax, ob):                                                        
    x, y = zip(*list((p.x, p.y) for p in ob.boundary))                          
    ax.plot(x, y, '.', color='#000000', zorder=1)                               
                                                                                
def plot_line(ax, ob):                                                          
    x, y = ob.xy                                                                
    ax.plot(x, y, color='cyan', alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)
                                                                                
def print_border(ax, waypoints, inner_border_waypoints, outer_border_waypoints):
    line = LineString(waypoints)                                                
    plot_coords(ax, line)                                                       
    plot_line(ax, line)                                                         
                                                                                
    line = LineString(inner_border_waypoints)                                   
    plot_coords(ax, line)                                                       
    plot_line(ax, line)                                                         
                                                                                
    line = LineString(outer_border_waypoints)                                   
    plot_coords(ax, line)                                                       
    plot_line(ax, line)
    
    line = LineString(outer_border_waypoints)                                   
    plot_coords(ax, line)                                                       
    plot_line(ax, line) 

## Generate outer and inner borders using shapely

In [ ]:
# Make a poly out of our center line, expanding its dimensions to have outer and inner boundaries
ls_center = LineString(waypoints)
poly = ls_center.buffer(TRACK_WIDTH/2.0)
print("Center Closed: ", ls_center.is_ring)
print("Center Length: ", ls_center.length)
print("Center Bounds: ", ls_center.bounds)
exterior_coords = poly.exterior.coords[:]
interior_coords = []
for interior in poly.interiors:
    interior_coords += interior.coords[:]

ls_outer = LineString(exterior_coords)
ls_inner = LineString(interior_coords)

#Generate final waypoint list as [center_x, center_y, inner_x, inner_y, outer_x, outer_y]
all_waypoints = list()
for i in range(len(ls_center.coords)):
    distance = i / len(ls_center.coords)
    center_point = ls_center.coords[i]
    outer_point = ls_outer.interpolate(distance, normalized=True)
    inner_point = ls_inner.interpolate(distance, normalized=True)
    all_waypoints.append(list(sum((center_point, inner_point.coords[0], outer_point.coords[0]), ())))

poly

## Print the Generated Track

In [ ]:
wp = np.array(all_waypoints)
center_line = wp[:,0:2]
inner_border = wp[:,2:4]
outer_border = wp[:,4:6]

fig = plt.figure(1, figsize=(16, 10))
ax = fig.add_subplot(111, facecolor='black')
plt.axis('equal')
print_border(ax, center_line, inner_border, outer_border)

In [ ]:
fname = "../tracks/%s.npy"% TRACK_NAME
np.save(fname, all_waypoints)

In [ ]:
# Check if the track is saved
available_track_files = glob.glob("../Tracks/**.npy")
available_track_names = list(map(lambda x: os.path.basename(x).split('.npy')[0], available_track_files))
available_track_names